In [19]:
import websocket
import json
import time
import threading
import uuid
import pandas as pd
from datetime import datetime
import logging

class TradingViewDataFetcher:
    def __init__(self, log_level=logging.WARNING):
        """
        Initialize the TradingView data fetcher with configurable logging.
        
        Args:
            log_level: Logging level (default: WARNING to suppress info messages)
        """
        logging.basicConfig(level=log_level, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)
        
        # Default WebSocket URL (can be changed if needed)
        self.ws_url = "wss://prodata.tradingview.com/socket.io/websocket"
        
        # Default headers
        self.headers = {
            "Origin": "https://www.tradingview.com",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    def fetch_data(self, symbol, exchange="NASDAQ", interval="D", num_bars=100, timeout=20, alt_urls=None):
        """
        Fetch historical price data from TradingView.
        
        Args:
            symbol: Trading symbol (e.g., "AAPL")
            exchange: Exchange name (e.g., "NASDAQ")
            interval: Time interval (e.g., "D" for daily, "W" for weekly, "M" for monthly, 
                     or "1" for 1 minute, "5" for 5 minutes, etc.)
            num_bars: Number of bars to request
            timeout: Maximum time to wait for data in seconds
            alt_urls: List of alternative WebSocket URLs to try if the primary fails
            
        Returns:
            pandas.DataFrame: DataFrame containing OHLCV data
        """
        urls_to_try = [self.ws_url]
        if alt_urls:
            urls_to_try.extend(alt_urls)
        
        for url in urls_to_try:
            self.logger.info(f"Trying WebSocket URL: {url}")
            
            # Reset state for each attempt
            self.session_id = f"qs_{uuid.uuid4().hex[:12]}"
            self.chart_session = f"cs_{uuid.uuid4().hex[:12]}"
            self.received_data = []
            self.error_occurred = False
            
            # Create and start WebSocket
            websocket.enableTrace(False)  # Set to True for verbose WebSocket logs
            ws = websocket.WebSocketApp(
                url,
                header=[f"{k}: {v}" for k, v in self.headers.items()],
                on_open=self._on_open(symbol, exchange, interval, num_bars),
                on_message=self._on_message,
                on_error=self._on_error,
                on_close=self._on_close
            )
            
            wst = threading.Thread(target=ws.run_forever)
            wst.daemon = True
            wst.start()
            
            try:
                # Wait for data
                start_time = time.time()
                while time.time() - start_time < timeout:
                    if self.error_occurred:
                        self.logger.error("Connection error detected, trying next URL if available")
                        break
                    
                    if len(self.received_data) >= num_bars/2:
                        self.logger.info(f"Received sufficient data: {len(self.received_data)} bars")
                        break
                        
                    time.sleep(0.5)
                
                # If we have data, process and return it
                if self.received_data:
                    ws.close()
                    return self._process_data()
                
            except KeyboardInterrupt:
                self.logger.info("Interrupted by user")
            finally:
                ws.close()
        
        # If we get here, all URLs failed
        self.logger.error("Failed to retrieve data from all WebSocket URLs")
        return None

    def _process_data(self):
        """Process the received data into a DataFrame."""
        df = pd.DataFrame(self.received_data)
        if not df.empty:
            df['Date'] = pd.to_datetime(df['Date'], unit='s')
            df = df.sort_values('Date')
            #df.set_index('Date', inplace=True)
            return df
        return None

    def _on_open(self, symbol, exchange, interval, num_bars):
        """Prepare the on_open callback with the required parameters."""
        def on_open_callback(ws):
            self.logger.info("WebSocket connection established")
            
            def run():
                try:
                    self._send_message(ws, "set_auth_token", ["unauthorized_user_token"])
                    time.sleep(0.2)
                    
                    self._send_message(ws, "chart_create_session", [self.chart_session, ""])
                    time.sleep(0.2)
                    
                    self._send_message(ws, "switch_timezone", [self.chart_session, "Etc/UTC"])
                    time.sleep(0.2)
                    
                    full_symbol = f"{exchange}:{symbol}"
                    symbol_payload = json.dumps({"symbol": full_symbol, "adjustment": "splits"})
                    self._send_message(ws, "resolve_symbol", [self.chart_session, "symbol_1", f"={symbol_payload}"])
                    time.sleep(0.2)
                    
                    self._send_message(ws, "create_series", [self.chart_session, "s1", "s1", "symbol_1", interval, num_bars, ""])
                    time.sleep(0.2)
                    
                    self._send_message(ws, "chart_create_study", [self.chart_session, "st1", "st1", "sds_1", full_symbol, interval, num_bars, ""])
                    
                    self.logger.info(f"Requested {num_bars} {interval} bars for {full_symbol}")
                except Exception as e:
                    self.logger.error(f"Error in initialization sequence: {e}")
            
            thread = threading.Thread(target=run)
            thread.daemon = True
            thread.start()
        
        return on_open_callback

    def _send_message(self, ws, method, params):
        """Send a properly formatted message to TradingView WebSocket."""
        try:
            data = json.dumps({"m": method, "p": params})
            message = f"~m~{len(data)}~m~{data}"
            ws.send(message)
            self.logger.debug(f"Sent: {method} - {params[:50]}{'...' if len(str(params)) > 50 else ''}")
        except Exception as e:
            self.logger.error(f"Error sending message {method}: {e}")

    def _on_message(self, ws, message):
        """Handle incoming WebSocket messages."""
        if message == "~h~":
            self.logger.debug("Received heartbeat, sending pong")
            ws.send("~h~")
            return
        
        if not message.startswith("~m~"):
            return
        
        try:
            msg_parts = message.split("~m~")
            for i in range(0, len(msg_parts)):
                if msg_parts[i] and msg_parts[i].isdigit() and i + 1 < len(msg_parts):
                    content = msg_parts[i+1]
                    if content and content.startswith("{"):
                        try:
                            data = json.loads(content)
                            
                            if isinstance(data, dict) and data.get("m") == "timescale_update":
                                self.logger.debug("Found timescale_update message")
                                payload = data.get("p", [])
                                if len(payload) > 1 and isinstance(payload[1], dict):
                                    for series_id, series_data in payload[1].items():
                                        if "s" in series_data and isinstance(series_data["s"], list):
                                            bars_count = len(series_data["s"])
                                            self.logger.debug(f"Found {bars_count} price bars")
                                            
                                            for bar in series_data["s"]:
                                                if "v" in bar and len(bar["v"]) >= 5:
                                                    ts, o, h, l, c = bar["v"][:5]
                                                    volume = bar["v"][5] if len(bar["v"]) > 5 else None
                                                    self.received_data.append({
                                                        "Date": ts,
                                                        "Open": o,
                                                        "High": h,
                                                        "Low": l,
                                                        "Close": c,
                                                        "Volume": volume
                                                    })
                        except json.JSONDecodeError:
                            self.logger.debug(f"Failed to parse JSON: {content[:50]}...")
        except Exception as e:
            self.logger.error(f"Error processing message: {e}")

    def _on_error(self, ws, error):
        """Handle WebSocket errors."""
        self.error_occurred = True
        self.logger.error(f"WebSocket error: {error}")

    def _on_close(self, ws, close_status_code, close_msg):
        """Handle WebSocket connection close."""
        self.logger.debug(f"WebSocket closed: {close_status_code} - {close_msg}")


# Example usage
def get_data(symbol, exchange="NASDAQ", interval="D", bars=5000, log_level=logging.WARNING):
    """
    Simple function to fetch data from TradingView.
    
    Args:
        symbol: Trading symbol (e.g., "AAPL")
        exchange: Exchange name (default: "NASDAQ")
        interval: Time interval (default: "D" for daily)
        bars: Number of bars to request (default: 5000)
        log_level: Logging level (default: WARNING - minimal output)
    
    Returns:
        pandas.DataFrame: DataFrame with OHLCV data or None if failure
    """
    fetcher = TradingViewDataFetcher(log_level=log_level)
    
    # Alternative URLs to try if the primary fails
    alt_urls = [
        "wss://data.tradingview.com/socket.io/websocket",
        "wss://s.tradingview.com/socket.io/websocket"
    ]
    
    return fetcher.fetch_data(
        symbol=symbol,
        exchange=exchange,
        interval=interval,
        num_bars=bars,
        alt_urls=alt_urls
    )
import re 





In [20]:
N_BARS = 6
expiries = 1
strikes = 1 
stocklist = ['AAPL', 'MSFT', 'TSLA']

In [21]:
import yfinance as yf
import pandas as pd
import re

def yahoo_to_tradingview(yahoo_symbol):
    """
    Convert Yahoo Finance option contract symbol to TradingView format,
    ensuring one decimal place (.0 or .5) for the strike.
    Example: 'AAPL250718C00190000' -> 'AAPL250718C190.0'
    """
    match = re.match(r'^([A-Z]+)(\d{6})([CP])(\d{8})$', yahoo_symbol)
    if not match:
        return None
    underlying, yymmdd, cp, strike_raw = match.groups()
    strike = int(strike_raw) / 1000
    # Format to always have one decimal place
    strike_str = f"{strike:.1f}"
    return f"{underlying}{yymmdd}{cp}{strike_str}"

  # Modify as needed
all_options = pd.DataFrame()

for symbol in stocklist:
    print(f"Fetching options for {symbol}")
    ticker = yf.Ticker(symbol)
    try:
        exchange = ticker.info.get('exchange', 'N/A')
    except Exception as e:
        print(f"Could not fetch exchange info for {symbol}: {e}")
        exchange = 'N/A'
    # Get current stock price
    try:
        # Use 'regularMarketPrice' for real-time price if available
        current_price = ticker.info.get('regularMarketPrice')
        if current_price is None:
            # Fallback: use last closing price
            current_price = ticker.history(period='1d')['Close'].iloc[-1]
    except Exception as e:
        print(f"Could not fetch current price for {symbol}: {e}")
        continue
    expirations = ticker.options[:expiries]  # Take only the 3 nearest expiries
    for exp_date in expirations:
        try:
            opt_chain = ticker.option_chain(exp_date)
            options_list = []
            for opt_df, opt_type in [(opt_chain.calls, 'call'), (opt_chain.puts, 'put')]:
                opt_df = opt_df.copy()
                opt_df['optionType'] = opt_type
                # Find 5 strikes closest to current price
                opt_df['strike_diff'] = (opt_df['strike'] - current_price).abs()
                opt_df = opt_df.nsmallest(strikes, 'strike_diff')
                options_list.append(opt_df)
            options = pd.concat(options_list, ignore_index=True)
            options['expirationDate'] = exp_date
            options['symbol'] = symbol
            options['exchange'] = exchange
            options['tradingview_symbol'] = options['contractSymbol'].apply(yahoo_to_tradingview)
            all_options = pd.concat([all_options, options], ignore_index=True)
        except Exception as e:
            print(f"Failed to fetch options for {symbol} {exp_date}: {e}")

#all_options = all_options.tail(3)
# all_options.to_csv("options_nearest_3_expiries_5_strikes.csv", index=False)  # Optional: save to CSV


Fetching options for AAPL
Fetching options for MSFT
Fetching options for TSLA


In [22]:
all_options

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,optionType,strike_diff,expirationDate,symbol,exchange,tradingview_symbol
0,AAPL250711C00212500,2025-07-03 16:59:59+00:00,212.5,3.35,3.30,3.40,0.100000,3.076920,18903.0,11612,0.226082,True,REGULAR,USD,call,1.05,2025-07-11,AAPL,NMS,AAPL250711C212.5
1,AAPL250711P00212500,2025-07-03 16:59:59+00:00,212.5,2.29,2.22,2.32,-0.750000,-24.671053,11946.0,0,0.223641,False,REGULAR,USD,put,1.05,2025-07-11,AAPL,NMS,AAPL250711P212.5
2,MSFT250711C00500000,2025-07-03 16:59:56+00:00,500.0,3.81,3.70,3.85,2.250000,144.230770,10359.0,4952,0.149362,False,REGULAR,USD,call,1.16,2025-07-11,MSFT,NMS,MSFT250711C500.0
3,MSFT250711P00500000,2025-07-03 16:59:49+00:00,500.0,4.73,4.70,4.80,-5.020000,-51.487180,2558.0,814,0.142220,True,REGULAR,USD,put,1.16,2025-07-11,MSFT,NMS,MSFT250711P500.0
4,TSLA250711C00315000,2025-07-03 16:59:59+00:00,315.0,8.65,8.70,8.80,-1.660001,-16.100880,16972.0,4017,0.463384,True,REGULAR,USD,call,0.35,2025-07-11,TSLA,NMS,TSLA250711C315.0
5,TSLA250711P00315000,2025-07-03 16:59:56+00:00,315.0,8.23,8.20,8.35,-1.120001,-11.978618,18444.0,2966,0.458013,False,REGULAR,USD,put,0.35,2025-07-11,TSLA,NMS,TSLA250711P315.0


In [23]:
import logging
import pandas as pd

# ... (your code for building all_options with 'tradingview_symbol') ...

def get_data(symbol, exchange="OPRA", interval="D", bars=3):
    """
    Simple function to fetch data from TradingView.
    """
    fetcher = TradingViewDataFetcher()
    alt_urls = [
        "wss://data.tradingview.com/socket.io/websocket",
        "wss://s.tradingview.com/socket.io/websocket"
    ]
    return fetcher.fetch_data(
        symbol=symbol,
        exchange=exchange,
        interval=interval,
        num_bars=bars,
        alt_urls=alt_urls
    )


all_options['tv_ohlcv_n'] = None

for idx, row in all_options.iterrows():
    tv_symbol = row['tradingview_symbol']
    try:
        ohlcv_df = get_data(tv_symbol, exchange="OPRA", interval="D", bars=N_BARS)
        if ohlcv_df is not None and not ohlcv_df.empty:
            all_options.at[idx, 'tv_ohlcv_n'] = ohlcv_df.tail(N_BARS)
        else:
            all_options.at[idx, 'tv_ohlcv_n'] = None
    except Exception as e:
        print(f"Failed to fetch TradingView data for {tv_symbol}: {e}")
        all_options.at[idx, 'tv_ohlcv_n'] = None

# Generate labels: [0, -1, -2, ..., -(N_BARS-1)]
labels = [0] + [-(i) for i in range(1, N_BARS)]

rows = []
for idx, row in all_options.iterrows():
    symbol = row['tradingview_symbol']
    ohlcv = row['tv_ohlcv_n']
    data = {'tradingview_symbol': symbol}
    if ohlcv is not None and not ohlcv.empty:
        ohlcv = ohlcv.tail(N_BARS).reset_index(drop=True)
        # Loop over bars and labels
        for i, label in zip(range(N_BARS-1, -1, -1), labels):
            if i < len(ohlcv):
                bar = ohlcv.iloc[i]
                for col in ['Open', 'Close', 'Volume']:
                    data[f'{col}_{label}'] = bar[col] if col in bar else None
            else:
                for col in ['Open', 'Close', 'Volume']:
                    data[f'{col}_{label}'] = None
    else:
        for label in labels:
            for col in ['Open', 'Close', 'Volume']:
                data[f'{col}_{label}'] = None
    rows.append(data)

option_bars_df = pd.DataFrame(rows)

# Display a sample of the new DataFrame
print(option_bars_df.head())


  tradingview_symbol  Open_0  Close_0  Volume_0  Open_-1  Close_-1  Volume_-1  \
0   AAPL250711C212.5    2.96     3.35   18903.0     2.06      3.25    28680.0   
1   AAPL250711P212.5    3.05     2.29   11946.0     5.90      3.04     5939.0   
2   MSFT250711C500.0    2.00     3.81   10359.0     1.57      1.56     5685.0   
3   MSFT250711P500.0    7.00     4.73    2558.0    11.81      9.75      201.0   
4   TSLA250711C315.0   11.25     8.65   16972.0     9.92     10.31    19370.0   

   Open_-2  Close_-2  Volume_-2  Open_-3  Close_-3  Volume_-3  Open_-4  \
0     1.62      1.67    19348.0     0.55      1.23     9278.0     0.60   
1     7.25      6.04      618.0    10.90      8.05      658.0    11.11   
2     3.85      2.06     3789.0     4.42      4.30     1893.0     5.15   
3     6.65      8.59      386.0     6.40      6.00     1434.0     6.65   
4     5.60      6.10     4628.0    16.10     13.60     1406.0    18.10   

   Close_-4  Volume_-4  Open_-5  Close_-5  Volume_-5  
0      0.50  

In [24]:
import logging
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
from datetime import datetime
import yfinance as yf

def black_scholes_price(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price

def implied_volatility(price, S, K, T, r, option_type):
    def objective(sigma):
        return black_scholes_price(S, K, T, r, sigma, option_type) - price
    try:
        return brentq(objective, 1e-6, 5)
    except:
        return np.nan

def black_scholes_greeks(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        delta = norm.cdf(d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
                 - r * K * np.exp(-r * T) * norm.cdf(d2)) / 365
    else:
        delta = -norm.cdf(-d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
                 + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 365
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100
    return delta, gamma, theta, vega

# Dynamically generate labels: [0, -1, -2, ..., -(N_BARS-1)]
N_BARS = 3
labels = [0] + [-(i) for i in range(1, N_BARS)]

rows = []
risk_free_rate = 0.05  # Example: 5% risk-free rate, adjust as needed

for idx, row in all_options.iterrows():
    symbol = row['tradingview_symbol']
    ohlcv = row['tv_ohlcv_n']
    option_type = row['optionType']
    strike = row['strike']
    expiry = row['expirationDate']
    bid = row.get('bid', np.nan)
    ask = row.get('ask', np.nan)
    open_interest = row.get('openInterest', np.nan)
    underlying = row['symbol']

    # Use mid price for IV calculation if possible
    if not np.isnan(bid) and not np.isnan(ask):
        price = (bid + ask) / 2
    elif not np.isnan(bid):
        price = bid
    elif not np.isnan(ask):
        price = ask
    else:
        price = np.nan

    # --- Add underlying stock open and low for each of the last N_BARS days ---
    stock_open_low_dict = {}
    stock_close_dict = {}
    S = np.nan  # Underlying price for Greeks/IV
    try:
        stock_hist = yf.Ticker(underlying).history(period=f"{N_BARS+10}d", interval="1d")  # extra days for non-trading days
        stock_hist = stock_hist.dropna(subset=['Open', 'Low', 'Close'])
        stock_hist = stock_hist.tail(N_BARS).reset_index(drop=True)
        for i, label in zip(range(N_BARS-1, -1, -1), labels):
            if i < len(stock_hist):
                stock_open_low_dict[f'stock_open_{label}'] = stock_hist.loc[i, 'Open']
                stock_open_low_dict[f'stock_low_{label}'] = stock_hist.loc[i, 'Low']
                stock_close_dict[label] = stock_hist.loc[i, 'Close']
            else:
                stock_open_low_dict[f'stock_open_{label}'] = None
                stock_open_low_dict[f'stock_low_{label}'] = None
                stock_close_dict[label] = None
        # Use the -1 label's close as S (yesterday's close)
        S = stock_close_dict.get(-1, np.nan)
    except Exception as e:
        for label in labels:
            stock_open_low_dict[f'stock_open_{label}'] = None
            stock_open_low_dict[f'stock_low_{label}'] = None
            stock_close_dict[label] = None

    # Calculate time to expiry in years
    try:
        expiry_dt = pd.to_datetime(expiry)
        today = pd.Timestamp(datetime.utcnow().date())
        T = (expiry_dt - today).days / 365.0
        if T <= 0:
            T = 1/365  # 1 day minimum to avoid div by zero
    except Exception as e:
        T = np.nan

    # Calculate IV and Greeks using S = close_-1 (yesterday's close)
    if not np.isnan(price) and not np.isnan(S) and not np.isnan(strike) and not np.isnan(T):
        try:
            iv = implied_volatility(price, S, strike, T, risk_free_rate, option_type)
        except Exception as e:
            iv = np.nan
        try:
            if not np.isnan(iv):
                delta, gamma, theta, vega = black_scholes_greeks(S, strike, T, risk_free_rate, iv, option_type)
            else:
                delta, gamma, theta, vega = [np.nan]*4
        except Exception as e:
            delta, gamma, theta, vega = [np.nan]*4
    else:
        iv, delta, gamma, theta, vega = [np.nan]*5

    # Extract OHLCV bars: label as 0, -1, -2, ..., -(N_BARS-1)
    ohlcv_dict = {}
    if ohlcv is not None and not ohlcv.empty:
        ohlcv.columns = [col.lower() for col in ohlcv.columns]
        ohlcv = ohlcv.tail(N_BARS).reset_index(drop=True)
        for i, label in zip(range(N_BARS-1, -1, -1), labels):
            if i < len(ohlcv):
                bar = ohlcv.iloc[i]
                for col in ['open', 'close', 'volume']:
                    ohlcv_dict[f'{col}_{label}'] = bar[col] if col in bar else None
            else:
                for col in ['open', 'close', 'volume']:
                    ohlcv_dict[f'{col}_{label}'] = None
    else:
        for label in labels:
            for col in ['open', 'close', 'volume']:
                ohlcv_dict[f'{col}_{label}'] = None

    data = {
        'tradingview_symbol': symbol,
        'bid': bid,
        'ask': ask,
        'openInterest': open_interest,
        'implied_volatility': iv,
        'delta': delta,
        'gamma': gamma,
        'theta': theta,
        'vega': vega
    }
    data.update(ohlcv_dict)
    data.update(stock_open_low_dict)
    rows.append(data)

option_bars_df = pd.DataFrame(rows)

# Display a sample of the new DataFrame
option_bars_df


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_54446/1651656773.py:97: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = pd.Timestamp(datetime.utcnow().date())
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_54446/1651656773.py:97: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = pd.Timestamp(datetime.utcnow().date())
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_54446/1651656773.py:97: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = pd.Timestamp(datetime.utcnow().dat

,tradingview_symbol,bid,ask,openInterest,implied_volatility,delta,gamma,theta,vega,open_0,...,volume_-1,open_-2,close_-2,volume_-2,stock_open_0,stock_low_0,stock_open_-1,stock_low_-1,stock_open_-2,stock_low_-2
0,AAPL250711C212.5,3.30,3.40,11612,0.279376,0.514690,0.048505,-0.248573,0.117288,2.96,...,28680.0,1.62,1.67,19348.0,212.149994,211.809998,208.910004,208.139999,206.669998,206.139999
1,AAPL250711P212.5,2.22,2.32,0,0.199544,-0.484725,0.067907,-0.152746,0.117282,3.05,...,5939.0,7.25,6.04,618.0,212.149994,211.809998,208.910004,208.139999,206.669998,206.139999
2,MSFT250711C500.0,3.70,3.85,4952,0.263902,0.327263,0.020110,-0.484206,0.245466,2.00,...,5685.0,3.85,2.06,3789.0,493.809998,493.440002,489.989990,488.700012,496.470001,490.980011
3,MSFT250711P500.0,4.70,4.80,814,NaN,NaN,NaN,NaN,NaN,7.00,...,201.0,6.65,8.59,386.0,493.809998,493.440002,489.989990,488.700012,496.470001,490.980011
4,TSLA250711C315.0,8.70,8.80,4017,0.474761,0.531408,0.019164,-0.611324,0.173848,11.25,...,19370.0,5.60,6.10,4628.0,317.989990,312.760010,312.630005,303.820007,298.459991,293.209991
5,TSLA250711P315.0,8.20,8.35,2966,0.502193,-0.468834,0.018118,-0.602233,0.173856,8.02,...,10156.0,21.86,20.00,2408.0,317.989990,312.760010,312.630005,303.820007,298.459991,293.209991
